# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [31]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [37]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

In [41]:
from sklearn.tree import DecisionTreeRegressor

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
spaceship.dropna(inplace=True)
spaceship.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [7]:
def reclassify_cabin(cabin):
    if cabin.startswith("A"):
        return "A"
    elif cabin.startswith("B"):
        return "B"
    elif cabin.startswith("C"):
        return "C"
    elif cabin.startswith("D"):
        return "D"
    elif cabin.startswith("E"):
        return "E"
    elif cabin.startswith("F"):
        return "F"
    elif cabin.startswith("G"):
        return "G"
    else:
        return "T"

spaceship["Cabin_Category"] = spaceship["Cabin"].apply(reclassify_cabin) 

In [9]:
spaceship.drop(columns = ["PassengerId", "Name"])

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_Category
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E


In [11]:
def reclassify_planet(planet):
    if planet == "Europa":
        return "1"
    elif planet == "Earth":
        return "2"
    else:
        return "3"

spaceship["Planet_Category"] = spaceship["HomePlanet"].apply(reclassify_planet)

In [13]:
def reclassify_cryosleep(cryosleep):
    if cryosleep == True:
        return "1"
    else:
        return "0"

def reclassify_transported(transported):
    if transported == True:
        return "1"
    else:
        return "0"

spaceship["cryosleep _cat"] = spaceship["CryoSleep"].apply(reclassify_cryosleep)
spaceship["transported_cat"] = spaceship["Transported"].apply(reclassify_transported)

In [15]:
features = spaceship.drop(columns = ["PassengerId", "HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Name", "Transported", "Cabin_Category", "transported_cat"])
features.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Planet_Category,cryosleep _cat
0,39.0,0.0,0.0,0.0,0.0,0.0,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,2,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,1,0
3,33.0,0.0,1283.0,371.0,3329.0,193.0,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,2,0


In [17]:
target = spaceship[("transported_cat")]
target.head()

0    0
1    1
2    0
3    0
4    1
Name: transported_cat, dtype: object

In [19]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
knn = KNeighborsClassifier(n_neighbors=3)

In [25]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [27]:
pred = knn.predict(X_test)
pred

array(['1', '1', '1', ..., '0', '0', '0'], dtype=object)

- Evaluate your model

In [33]:
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", knn.score(X_test, y_test))

MAE 0.24130105900151286
RMSE 0.4912240415548824
R2 score 0.7586989409984871


C:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [43]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [45]:
ada_reg.fit(X_train, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [47]:
pred = ada_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test, y_test))

MAE 0.2895477841336338
RMSE 0.44551088857819415
R2 score 0.20384801293891652


C:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [49]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [51]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())

In [57]:
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5)

In [59]:
model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [10, 30, 50],
                         'estimator__max_leaf_nodes': [250, 500, 1000, None],
                         'n_estimators': [50, 100, 200, 500]})

In [61]:
model.best_params_

{'estimator__max_depth': 10,
 'estimator__max_leaf_nodes': 250,
 'n_estimators': 50}

- Run Grid Search

In [63]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                            n_estimators=50)

In [65]:
ada_reg.fit(X_train, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=10))

In [67]:
pred = ada_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test, y_test))

MAE 0.3617789546443557
RMSE 0.4226382836741026
R2 score 0.2834986623608402


C:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Evaluate your model